## Import library

In [1]:
# importing required libraries
import os, sys
import sys
sys.path.insert(1, '/home/dtd/Documents/interpretable_machine_learning/Source Code/my_work/lib')
sys.path.insert(1, '/home/dtd/Documents/interpretable_machine_learning/Causal Inference/CEA')
import loader


import numpy as np
import pandas as pd
import logging
import dowhy.datasets
import dowhy
import propensity_score_estimator as pse
import incremental_ps_score_estimator as ipse
import math 
import timeit
import evaluation as evl

from scipy.stats import sem
from dowhy import CausalModel
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from math import sqrt
from econml.dml import LinearDMLCateEstimator
from sklearn.linear_model import LassoCV
from econml.inference import BootstrapInference
from econml.dml import SparseLinearDMLCateEstimator
from sklearn.linear_model import LassoCV, ElasticNetCV
from sklearn.ensemble import GradientBoostingRegressor

/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


## Do experiment on EconML package

In [2]:
PATH_TRAIN = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-1000.train.npz"
PATH_TEST = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-1000.test.npz"

In [3]:
def load_data(path, i_exp):
    col =  ["t", "yf", "ycf", "mu0", "mu1" ]
    cov = ["x" + str(i) for i in range(1,26)]
    col = col + cov
    features = cov + ["t"] 
    D = np.load(path)
    df = pd.DataFrame(columns=col)

    for i in range(1,26):
        df['x' + str(i)]  = D['x'][:,i-1,i_exp-1]

    df['t']  = D['t'][:,i_exp-1:i_exp]
    df['yf'] = D['yf'][:,i_exp-1:i_exp]
    df['ycf'] = D['ycf'][:,i_exp-1:i_exp]
    df['mu0'] = D['mu0'][:,i_exp-1:i_exp]
    df['mu1'] = D['mu1'][:,i_exp-1:i_exp]
    return df

In [4]:
treatment = 't'
outcome = 'yf'
col =  ["t", "yf", "ycf", "mu0", "mu1" ]
cov = ["x" + str(i) for i in range(1,26)]
col = col + cov
features = cov + ["t"] 

In [5]:
linear_mae = []
forest = []

for i in tqdm(range(10,11)):    
    data_train = load_data(PATH_TRAIN, i)
    data_test  = load_data(PATH_TEST, i)
    
    true_effect = data_test['mu1'] - data_test['mu0']
    
    est = LinearDMLCateEstimator(model_y=ElasticNetCV(), model_t=ElasticNetCV())
    
    est.fit(Y = data_train[outcome].values, 
            T = data_train[treatment].values, 
            X = data_train[cov].values, 
            inference='statsmodels')
    treatment_effects = est.effect(data_test[cov])
    print(treatment_effects)
    mae = evl.mae_ate(treatment_effects, true_effect)
    linear_mae.append(mae)

    
    '''
    est = ForestDMLCateEstimator(model_y=GradientBoostingRegressor(), model_t=GradientBoostingRegressor())
    est.fit(Y = data_train[treatment], 
        T = data_train[outcome], 
        X = data_train[cov], 
        inference='blb')
    treatment_effects = est.effect(X_test)
    mae = evl.mae_ate(treatment_effects, true_effect)
    forest.append(mae)
    '''
    
    
    
    
    

  0%|          | 0/1 [00:00<?, ?it/s]/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.2511014454858014, tolerance: 2.478003210511227
  positive)
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.635709372459587, tolerance: 2.478003210511227
  positive)
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.248090033892595, tolerance: 1.8358351675199787
  positive)
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: Convergence

[ -3.2238633   -6.19863168   3.95031316  -1.68458458  -2.01125372
  -7.28387911  13.60234571  11.7181219   -0.84396941  -7.1544296
   2.00043393  12.00577817  -2.94061815  13.21213987   7.18715017
  10.69342547  -2.36253958 -12.03126682  -2.74198408  10.40511504
  18.21882182 -14.55498851   1.21753822   0.45523833  13.36148209
  12.42765173   7.42249094  -1.9605218    9.41362543  18.21173345
  22.39221904  11.37767777  -3.4389171   -6.21398006   2.95211668
  -4.43922194  -1.83534667   8.46874724  -4.06307937  18.62890379
   9.29142447 -13.63695973   7.82809108  15.99612134  -5.01278852
   4.1679607    1.49316276   3.07502446  11.0799004   -0.16351892
  -9.51980413 -14.1277703   -5.88558525  -2.49863312  10.47200464
  -7.43810317  -5.07546759  -4.47968557  10.22531538   3.17245583
 -14.55558232  -3.18290712  -1.99483899  -5.51901402   8.09788296
  -1.03315877   0.44163729   5.59963761  -5.80162602  -1.78079147
  -6.79241016   6.20534946   3.77337668  24.36691856  -1.65000304]


In [6]:
means, stds = np.mean(treatment_effects, axis=0), sem(treatment_effects, axis=0)
print('Linear Effect: {:.3f}+-{:.3f}'.format(means, stds))

means, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)
print('True effect: {:.3f}+-{:.3f}'.format(means, stds))


Linear Effect: 2.126+-1.034
True effect: 2.779+-0.981


## Do experiement on z-estimator

In [7]:
# z_estimator_mae = []
# delta_seq = np.linspace(0.5, 10, 50)
# for i in tqdm(range(9,10)):
#     data_train = load_data(PATH_TRAIN, i)
#     data_test = load_data(PATH_TEST, i)
#     estimator = 'z_estimator'
#     mae = ipse.experiment_z_estimator_train_test(data_train, data_test, treatment, cov, outcome, estimator, delta_seq)
#     z_estimator_mae.append(mae)
#     del data_train, data_test

In [8]:
# means, stds = np.mean(z_estimator_mae, axis=0), sem(z_estimator_mae, axis=0)
# print('Linear MAE: {:.3f}+-{:.3f}'.format(means, stds))

In [9]:
# z_estimator_mae

In [10]:
data_train = load_data(PATH_TRAIN, 10)
data_test  = load_data(PATH_TEST,10)

delta = 2
features = cov.copy()
features.append(treatment)

## Fit treatment
model_t = LogisticRegression()
model_t.fit(data_train[cov], data_train[treatment])

## Fit outcome
model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
model_y.fit(data_train[features], data_train[outcome])



## Compute propensity score
data_test['ps_1'] = model_t.predict_proba(data_test[cov])[:,1]
data_test['ps_0'] = 1 - data_test['ps_1']

## fit outcome
data_test['predicted_y'] = model_y.predict(data_test[features])
## Compute counterfactual outcome with no treatment
data_pos = data_test.copy()
data_pos[treatment] = 1
data_test['treated_cf_outcome'] = model_y.predict(data_pos[features])

## Compute counterfactual outcome with treatment
data_neg = data_test.copy()
data_neg[treatment] = 0
data_test['control_cf_outcome'] = model_y.predict(data_neg[features])

count = 0
effect = []

## Compute incremental score 
data_test['incre_ps_1'] = (delta * data_test['ps_1']) / (delta * data_test['ps_1'] + data_test['ps_0'])
data_test['incre_ps_0'] = 1 - data_test['incre_ps_1']

## Construct time-dependent weights
w_t = (delta*data_test[treatment] + 1 - data_test[treatment]) / (delta*data_test['ps_1'] + data_test['ps_0'])

v_t = (data_test[treatment]*data_test['ps_0']
                - (1 - data_test[treatment])*data_test['ps_1']*delta) / (delta / (1-delta))

## Construct pseudo-outcome Rt
r_t = (data_test['treated_cf_outcome']*data_test['incre_ps_1'] + 
       data_test['control_cf_outcome']*data_test['incre_ps_0'])

estimation_effect = w_t*data_test[outcome] + w_t*v_t*r_t

In [17]:
data_test.head()

,t,yf,ycf,mu0,mu1,x1,x2,x3,x4,x5,...,x23,x24,x25,ps_1,ps_0,predicted_y,treated_cf_outcome,control_cf_outcome,incre_ps_1,incre_ps_0
0,0.0,21.577024,23.450651,22.032736,24.868509,-0.720986,-0.602710,1.500917,1.203011,-0.191569,...,0.0,1.0,0.0,0.054239,0.945761,23.713070,24.000010,23.713070,0.102897,0.897103
1,0.0,33.170801,26.747131,32.060751,25.243612,-1.088461,-0.602710,0.383828,-0.879606,1.058775,...,0.0,0.0,0.0,0.215118,0.784882,31.822857,28.354603,31.822857,0.354069,0.645931
2,0.0,18.692773,25.810269,17.268386,24.624857,-0.331896,-0.202946,0.756191,0.161703,0.996258,...,0.0,0.0,0.0,0.049540,0.950460,15.819234,19.155036,15.819234,0.094403,0.905597
3,0.0,22.606214,26.484153,22.872999,24.905937,0.921840,0.596582,-0.360898,-0.879606,-1.066810,...,0.0,0.0,0.0,0.397547,0.602453,21.377041,22.491699,21.377041,0.568921,0.431079
4,0.0,23.341006,25.457547,25.438503,25.012244,1.181234,1.396110,-0.360898,2.244320,-0.379120,...,0.0,0.0,0.0,0.235533,0.764467,23.347927,24.739313,23.347927,0.381266,0.618734


In [16]:
means, stds = np.mean(treatment_effects, axis=0), sem(treatment_effects, axis=0)
print('Linear Effect: {:.3f}+-{:.3f}'.format(means, stds))

means, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)
print('True effect: {:.3f}+-{:.3f}'.format(means, stds))

estimation_effect = w_t*r_t + w_t*v_t*r_t
means, stds = np.mean(estimation_effect, axis=0), sem(estimation_effect, axis=0)
print('Z-estimator effect: {:.3f}+-{:.3f}'.format(means, stds))

estimation_effect = w_t*data_test[outcome] + w_t*v_t*r_t
means, stds = np.mean(estimation_effect, axis=0), sem(estimation_effect, axis=0)
print('Z-estimator effect: {:.3f}+-{:.3f}'.format(means, stds))

estimation_effect = w_t*(data_test['treated_cf_outcome'] - data_test['control_cf_outcome'])
means, stds = np.mean(estimation_effect, axis=0), sem(estimation_effect, axis=0)
print('Z-estimator effect: {:.3f}+-{:.3f}'.format(means, stds))

estimation_effect = w_t*v_t*r_t
means, stds = np.mean(estimation_effect, axis=0), sem(estimation_effect, axis=0)
print('Z-estimator effect: {:.3f}+-{:.3f}'.format(means, stds))

Linear Effect: 2.126+-1.034
True effect: 2.779+-0.981
Z-estimator effect: 23.187+-0.719
Z-estimator effect: 23.044+-0.866
Z-estimator effect: 3.354+-0.898
Z-estimator effect: -0.954+-1.031


## Inverse propability schore matching

In [59]:
num_units = data_test.shape[0]
data_test['ips_weight'] = (1/num_units) * (
    data_test[treatment] / data_test['ps_1'] + (1 - data_test[treatment]) / (1 - data_test['ps_1'])
)

# Calculating the effect
data_test['d_y'] = (
    data_test['ips_weight'] *
    data_test[treatment] *
    data_test[outcome]
)
data_test['dbar_y'] = (
    data_test['ips_weight'] *
    (1 - data_test[treatment]) *
    data_test[outcome]
)
est = data_test['d_y'] - data_test['dbar_y']

## Augmented Inverse Probability Weighted Estimator (AIPWE)


In [21]:
data_test.head()

,t,yf,ycf,mu0,mu1,x1,x2,x3,x4,x5,...,x23,x24,x25,ps_1,ps_0,predicted_y,treated_cf_outcome,control_cf_outcome,incre_ps_1,incre_ps_0
0,0.0,21.577024,23.450651,22.032736,24.868509,-0.720986,-0.602710,1.500917,1.203011,-0.191569,...,0.0,1.0,0.0,0.054239,0.945761,23.713070,24.000010,23.713070,0.102897,0.897103
1,0.0,33.170801,26.747131,32.060751,25.243612,-1.088461,-0.602710,0.383828,-0.879606,1.058775,...,0.0,0.0,0.0,0.215118,0.784882,31.822857,28.354603,31.822857,0.354069,0.645931
2,0.0,18.692773,25.810269,17.268386,24.624857,-0.331896,-0.202946,0.756191,0.161703,0.996258,...,0.0,0.0,0.0,0.049540,0.950460,15.819234,19.155036,15.819234,0.094403,0.905597
3,0.0,22.606214,26.484153,22.872999,24.905937,0.921840,0.596582,-0.360898,-0.879606,-1.066810,...,0.0,0.0,0.0,0.397547,0.602453,21.377041,22.491699,21.377041,0.568921,0.431079
4,0.0,23.341006,25.457547,25.438503,25.012244,1.181234,1.396110,-0.360898,2.244320,-0.379120,...,0.0,0.0,0.0,0.235533,0.764467,23.347927,24.739313,23.347927,0.381266,0.618734


In [43]:
w0 = np.where(data_test[treatment] == 0, 1, 0)
w1 = np.where(data_test[treatment] == 1, 1, 0)

In [49]:
m0 = (data_test[outcome]*w0 / data_test['ps_1']) - ((1 - data_test['ps_1']) / data_test['ps_1'])*data_test['control_cf_outcome']
m1 = (data_test[outcome]*w1 / data_test['ps_1']) - ((1 - data_test['ps_1']) / data_test['ps_1'])*data_test['treated_cf_outcome']